In [ ]:
import json
import sys
import threading
import websocket
import simplejson
from time import sleep
from datetime import datetime
import logging
import os
from RofexAPI import PJS_RESTv3 as pjs
from RofexAPI import MDmessages as md
sys.path.append('C:/Users/pauli/Documents/python/OptionModelsV3/RofexAPI')


In [ ]:
logging.basicConfig()

In [ ]:
#test funcion datetime
year = str(datetime.today().year)
day = str(datetime.today().day)
m=0
m+=1
if(len(day)==1):
    day = "0"+day
month = str(datetime.today().month)
if(len(month)==1):
    month = "0"+month
print(year)
print(month)
print(day)
print (m)

In [ ]:
listaMensajes = []
subsMDEvents = []
subsOREvents = []
mensajes=0
MSG_MDSuscription = '{{"type":"smd","level":1,"entries":[{e}],"products":[{s}]}}'
MSG_OSSuscription = '{{"type":"os","account":{{"id":"{a}"}}}}'
#MSG_Symbol = '{{"symbol":"{s}","marketId":"ROFX"}}'
MSG_Double_Quotes = '"{s}"'
#global mensajes

In [ ]:
def on_message(ws, message):
   # now  = datetime.now()
        
    try:
        print ("en el try")
        # Valido Mensaje entrante
        msg = simplejson.loads(message)
        msgType = msg['type'].upper()


        if msgType == 'MD':
            md.MDincoming(msg)
            
            # Aca es Market Data, esta escuchando novedades
            
            #subsMDEvents.append(msg)
            print("----------------------------------------------")
            #print("Mensajes:---->", len(subsMDEvents))
            global mensajes
            mensajes+=1
            print("Mensajes:---->", mensajes)
            
            """
            print ("En mensaje MD")
            #sys.stdout.flush()
            now  = datetime.now()
            print(str(now), msg)
           
                      
            
            #print("before processinf msg:...")
            symbol=msg['instrumentId']['symbol']
            
            #print (symbol)
            
            #Aca hay un problema si no hay bid u offer pq solo viene ['marketData']
            bidMsg=msg['marketData']['BI']
            offerMsg=msg['marketData']['OF']
            #print("BID msg :", bidMsg)
            #print("OFFER msg :",offerMsg)
            if bidMsg==[]:
                print("------------------------------------------>No BID detected")
                bid=0
                bidSize=0
            else:
                bid=msg['marketData']['BI'][0]['price']
                bidSize=msg['marketData']['BI'][0]['size']
            if offerMsg==[]:
                print("------------------------------------------->No OFFER detected")
                offer=0
                offerSize=0
            else:
                offer=msg['marketData']['OF'][0]['price']
                offerSize=msg['marketData']['OF'][0]['size']
                       
            
            msgArray=[]          
            msgArray.append(symbol)
           
            msgArray.append(bid)
            msgArray.append(bidSize)
            msgArray.append(offer)
            msgArray.append(offerSize)
            listaMensajes.append(msgArray)
           
            
            print(symbol,"----->", bid,"/", offer," SIZE--> ",bidSize,"/",offerSize)
            print("------------------------------------------------------")
            
                       
            
            if (bidSize ==7):
                print("BID Size of ", symbol,"----------------->",bidSize)
            
                #mensajes=0
                
            
            
            #si queremos escribir a un archivo usamos algo asi:
            #filename.write(str(now)+"|"+str(data['marketData']['LA']['price'])+"|"+str(data['marketData']['LA']['size'])+"|"+str(data['marketData']['BI'][0]['price'])+"|"+str(data['marketData']['BI'][0]['size'])+"|"+str(data['marketData']['OF'][0]['price'])+"|"+str(data['marketData']['OF'][0]['size'])+"\n");
            #filename.close()
            """
            
        elif msgType == 'OR':
            print ("En Mensaje OR")
            print(msg)
        else:
            print ("Tipo de Mensaje Recibido No soportado: " + msg)
    except:
        print ("Error al procesar mensaje recibido: " + message)

def on_error(ws, error):
    print ("Salio por error")
    print (error)
    ws.close()
    
def on_close(ws):
    print ("### connection closed ###")
    
def on_open(ws):    
    
    print ("On Conection Open...")
    

In [ ]:
def initWS():
    # 1st set urls
    # 2nd user data 
    entorno=1
    pjs.setURLs()
    pjs.init("balanz","balanz","555",entorno)
    
    #parametros contratos a suscribir
    type_="smd"
    level_="1"
    marketId_="ROFX"
       
    
    symbols=pjs.suscribirContratos()
    print("Contratos suscriptos:")
    for s in symbols:
        print(s)
    print('\n')
    
    
    isLogin = pjs.login()
    if isLogin:
        
        # control
        print("Conexion establecida...")
        print('Token:'+' '+pjs.token)
        # end control
        
        headers = {'X-Auth-Token:{token}'.format(token=pjs.token)}

        ws = websocket.WebSocketApp(pjs.activeWSEndpoint,
                                         on_message=on_message,
                                         on_error=on_error,
                                         on_close=on_close,
                                         on_open=on_open,
                                         header=headers)

        wst = threading.Thread(target=ws.run_forever, kwargs={"ping_interval" :5})
        
        wst.start()

        # Esperamos a que la conexion ws se establezca
        conn_timeout = 5
        #conn_timeout = 50 #y nada
        sleep(1)


        while not ws.sock.connected and conn_timeout:
            sleep(1)
            conn_timeout -= 1
        else:
            
        
            for sym in symbols:
                
                aaa=pjs.buildMessage(type_,level_,marketId_,sym)
                ws.send(aaa)
                
                print("Sent",aaa)
                           
                print("Receiving...")
                
                sleep(1) #y nada

    else:
        print( "Error al autenticarse...")


if __name__ == "__main__":
    initWS()
    